In [1]:
import leafmap
import numpy as np
import requests
import pds4_tools
from ipyleaflet import Map, WMSLayer, LayersControl, Polygon, GeoData
import ipyleaflet
import pandas as pd
import io
import pds.peppi as pep

# Step 1: Fetch MESSENGER MDIS image data from pds4

### Set and format parameters for request. 

Note: the below parameters were found by observing the HTTPS links below with a web json formatter and identifying which search parameters would be useful in the notebook. There are many more to choose from for each data set!

https://pds.mcp.nasa.gov/api/search/1/products/urn:nasa:pds:messenger_mdis_4001:browse::1.0/members


### Load the 1001 collection of elevation models

The following block extracts the browse and data collection's product and joins them together. 

Note: by an extra-ordinary circumptance, for collection 1001, none of the ref_lid_data in the browse collection matches the lid of the data product collection.

In [ ]:
client = pep.PDSRegistryClient()
products = pep.Products(client)


north = 'cart:Bounding_Coordinates.cart:north_bounding_coordinate'
east = 'cart:Bounding_Coordinates.cart:east_bounding_coordinate'
south = 'cart:Bounding_Coordinates.cart:south_bounding_coordinate'
west = 'cart:Bounding_Coordinates.cart:west_bounding_coordinate'

fname_browse = ['ops:Data_File_Info.ops:file_ref', 'ref_lid_data']
fname_prod = [
    east,north,west,south,
    'lid',
    'ops:Label_File_Info.ops:file_size',
    'ops:Data_File_Info.ops:creation_date_time',
    'ops:Data_File_Info.ops:file_ref'
]

df_browse = products.of_collection('urn:nasa:pds:messenger_mdis_dem_1001:browse::1.0').fields(fname_browse).as_dataframe()

# TODO a bug requires to re-initialize the product object, see https://github.com/NASA-PDS/peppi/issues/79
products = pep.Products(client)
df_prod = products.of_collection('urn:nasa:pds:messenger_mdis_dem_1001:elev::1.0').fields(fname_prod).as_dataframe()

df_1001 = df_prod.join(
    df_browse.set_index("ref_lid_data"),
    on="lid",
    lsuffix='_prod', rsuffix='_browse'
)

# Step 2: Visualize data on basemap

### Convert pandas dataframes into geodata geometry
The product information is viewable in an HTML widget using the geojson function in leafmap. The below code embeds the image URLS and product HTTPS into text that can be interacted with in the widget. 

In [8]:
from shapely import Point, LineString, Polygon
import geopandas as gpd

def polygon(row):
    if row[east] != 'null' and row[south] != 'null' and row[west] != 'null' and row[north] != 'null':
        return Polygon([
            (row[east], row[south]),
            (row[west], row[south]),
            (row[west], row[north]),
            (row[east], row[north]),
            (row[east], row[south])
        ])
    else:
        return None

def shortened_product_id(row):
    return row["lid"].split(":")[-1]

def data_product_metadata(row):
    return f"<a target='_blank' href='https://pds.nasa.gov/api/search/1/products/{row['lid']}'> View in Browser </a>"
    
def browse_product_link(row):
    browse_product_link = row['ops:Data_File_Info.ops:file_ref_browse']
    if isinstance(browse_product_link, str):
        return f"<a  target='_blank' href='{row['ops:Data_File_Info.ops:file_ref_browse']}'> View </a>"
    else:
        return "N/A"


def data_product_link(row):
    for data_link in row['ops:Data_File_Info.ops:file_ref_prod']:
        if data_link.endswith(".TAB") or data_link.endswith(".IMG") :
            return f"<a  target='_blank' href='{data_link}'> Download </a>"
    return "N/A"


df_1001 = df_1001[3:] #skip the first 2 products because we do not want to visualize these images
#Sort the values based on the east longitude coordinate - this is for the sake of layer visibility
df_1001 = df_1001.sort_values('cart:Bounding_Coordinates.cart:east_bounding_coordinate', ascending = False)
polygons = df_1001.apply(polygon, axis=1)
gdf_1001 = gpd.GeoDataFrame(geometry=polygons)
gdf_1001["product_id"] =  df_1001.apply(shortened_product_id, axis=1)
gdf_1001["product_metadata"] = df_1001.apply(data_product_metadata, axis=1)
gdf_1001["product_quicklook"] = df_1001.apply(browse_product_link, axis=1)
gdf_1001["product_data"] = df_1001.apply(data_product_link, axis=1)




### Instantiate basemap layers, convert geo data frames to geojson data and add them to the leafmap layer

In [9]:
wmsLayer = WMSLayer(
    url='https://planetarymaps.usgs.gov/cgi-bin/mapserv?map=/maps/mercury/mercury_simp_cyl.map',
    layers='MESSENGER',
    name='Messenger Basemap',
    crs=ipyleaflet.projections.EPSG4326
)

merc_map = leafmap.Map(layers=(wmsLayer, ), center=(0, 0), zoom=4, crs = ipyleaflet.projections.EPSG4326)
merc_map.add_control(LayersControl())


style = {
    "stroke": True,
    "color": "#ff0000",
    "weight": 2,
    "opacity": 1,
    "fill": True,
    "fillColor": "#ff0000",
    "fillOpacity": 0.1,
}

hover_style = {"fillOpacity": 0.7}

gdf1001_geojson = leafmap.gdf_to_geojson(gdf_1001, epsg="4326")
merc_map.add_geojson(gdf1001_geojson, layer_name="1001 Collection", style = style, hover_style = hover_style)

merc_map

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…